In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-30 10:49:05--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2026-01-30 10:49:05 (105 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt','r',encoding = 'UTF-8') as f:
  text = f.read()

In [ ]:
len(text)

1115394

In [ ]:
chars = sorted(list(set(text)))
vocab_size = 65

In [ ]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
n = int(0.9*len(data))

In [ ]:
train_data = data[:n]
test_data = data[n:]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
eval_iters = 200


def get_batch(split):

  data = train_data if split == 'train' else test_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])

  x, y = x.to(device), y.to(device)

  return x, y



@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','test']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


class Head(nn.Module):
  # only 1 head of attention

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias = False)
    self.value = nn.Linear(n_embd, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):

    B,T,C = x.shape
    k = self.key(x)   #(B,T,C)
    q = self.query(x)  #(B,T,C)

    wei = q @ k.transpose(-2,-1)*C**-0.5   #(B,T,C) x (B,T,C) => k->(B,T,C)->(B,C,T) => wei=(B,T,C)
    wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
    wei = F.softmax(wei, dim = -1)
    wei = self.dropout(wei)

    v = self.value(x)
    out = wei @ v    # (B, T, T) @ (B, T, C) -> (B, T, C)
    return out

class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList(Head(head_size) for _ in range(num_heads))
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim = -1)
    out = self.dropout(self.proj(out))
    return out


class FeedForward(nn.Module):

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd),
        nn.Dropout(dropout)
        )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):

      super().__init__()
      head_size = n_embd // n_head
      self.sa = MultiHeadAttention(n_head, head_size)
      self.ffwd = FeedForward(n_embd)
      self.ln1 = nn.LayerNorm(n_embd)
      self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
      x = x + self.sa(self.ln1(x))
      x = x + self.ffwd(self.ln2(x))
      return x

class BiGramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):

      idx_cond = idx[:,-block_size:]

      logits, loss = self(idx_cond)

      logits = logits[:, -1, :]

      probs = F.softmax(logits, dim = -1)

      idx_next = torch.multinomial(probs, num_samples = 1)

      idx = torch.cat((idx, idx_next), dim = 1)

    return idx

In [ ]:
model = BiGramLanguageModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)


for iter in range(max_iters):

  if iter % eval_iters == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['test']:.4f}")

  xb, yb = get_batch('train')

  logits,loss = model(xb, yb)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

context = torch.zeros((1,1), dtype = torch.long, device = device)

print(decode(m.generate(context, max_new_tokens = 2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.3129, val loss 4.3161
step 200: train loss 2.5064, val loss 2.5187
step 400: train loss 2.3650, val loss 2.3702
step 600: train loss 2.2690, val loss 2.2764
step 800: train loss 2.1756, val loss 2.1988
step 1000: train loss 2.0997, val loss 2.1373
step 1200: train loss 2.0424, val loss 2.0919
step 1400: train loss 2.0071, val loss 2.0729
step 1600: train loss 1.9594, val loss 2.0451
step 1800: train loss 1.9181, val loss 2.0043
step 2000: train loss 1.9053, val loss 1.9999
step 2200: train loss 1.8669, val loss 1.9711
step 2400: train loss 1.8435, val loss 1.9572
step 2600: train loss 1.8218, val loss 1.9528
step 2800: train loss 1.8108, val loss 1.9373
step 3000: train loss 1.7912, val loss 1.9161
step 3200: train loss 1.7619, val loss 1.9004
step 3400: train loss 1.7552, val loss 1.8947
step 3600: train loss 1.7523, val loss 1.8991
step 3800: train loss 1.7322, val loss 1.8778
step 4000: train loss 1.7084, val loss 1.8537
step 4200: train lo

In [ ]:
print(decode(m.generate(context, max_new_tokens = 2000)[0].tolist()))


Thou sumpeo desoneds or earshict thy while him hearts Onober,
Why fair Solty'll themself heaven:
I chink doth's see extry of Clorives to rigns?

Nor go will now sir make: your pentaking exse.
Titrance sore of not lot; King How thus on Cloinslain; did thy brinkes-- Isberned their any than messe
That which is regnives! whoulss not, come as sinces! scarrtune.

POLIxET:

ANGELO:
Prines shall Moman of them ben sir,' be my reash I satten's thou not shall fays.

EXENIUS:
Lord body trief: Myself. Mark'd, you when stralttence spery.
! First,--
Or bookingeloss, enentinging Lord him their pass!

RICKINGHARD:
What shall I sking to the fatom;
As that is no sonle dutten her shall bedate
Whet of my starge our vealione, death himsel
With O say, she kears nowerw'st he tates,
Which not made, if the persentedge thou him nor rewititness of Nisment.

LEONTEGSS That my tone:
Thou dekiss be ot marry.

ISABELLA:
A repence a pardons of Mower'd?

RUMENEENIS:
To the womblesan are us I tou, so, my in this went t